# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christianschroeder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christianschroeder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/christianschroeder/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages', engine)

# fetch the message feature as input
X = df['message']
# use all other features as labels, while dropping 'orginal' (ie. message in native language), and 'genre'
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)

In [26]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [27]:
X.isnull().values.any()

False

In [28]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
Y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 2. Write a tokenization function to process your text data

In [30]:
lemmatizer = WordNetLemmatizer()

In [31]:
def tokenize(text: str) -> list:
    words = [word for word in word_tokenize(text.lower()) if word.isalnum()]
    return [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]

In [32]:
tokenize(X[7])

['please', 'need', 'tent', 'water', 'silo', 'thank']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
rfc = RandomForestClassifier(random_state=42, min_samples_split=7)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(rfc, n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x14e4a28b0>)),
                ('tfidf', TfidfTransformer()),
                ('mclf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=7,
                                                                        random_state=42),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
def print_scores(y_test, y_pred):
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, i]))
    accuracy = (y_pred == y_test.values).mean()
    print('Model accuracy is {:.3f}'.format(accuracy))

In [36]:
y_pred = pipeline.predict(X_test)
print_scores(y_test, y_pred)

related
              precision    recall  f1-score   support

         0.0       0.38      0.06      0.10      2096
         1.0       0.76      0.97      0.85      6497

    accuracy                           0.75      8593
   macro avg       0.57      0.51      0.48      8593
weighted avg       0.67      0.75      0.67      8593

request
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.90      7117
         1.0       0.41      0.04      0.08      1476

    accuracy                           0.83      8593
   macro avg       0.62      0.52      0.49      8593
weighted avg       0.76      0.83      0.76      8593

offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      8547
         1.0       0.00      0.00      0.00        46

    accuracy                           0.99      8593
   macro avg       0.50      0.50      0.50      8593
weighted avg       0.99      0.99      0.99      859

/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8489
         1.0       0.00      0.00      0.00       104

    accuracy                           0.99      8593
   macro avg       0.49      0.50      0.50      8593
weighted avg       0.98      0.99      0.98      8593

refugees
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      8318
         1.0       0.00      0.00      0.00       275

    accuracy                           0.97      8593
   macro avg       0.48      0.50      0.49      8593
weighted avg       0.94      0.97      0.95      8593

death
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      8231
         1.0       0.00      0.00      0.00       362

    accuracy                           0.96      8593
   macro avg       0.48      0.50      0.49      8593
weighted avg       0.92      0.96      0.94      8593

othe

/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
rfc = RandomForestClassifier(random_state=42)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(estimator=rfc, n_jobs=-1))
])

parameters = {
    'vect__max_df': (0.8, 0.9),
    'mclf__estimator__max_depth': [5, 50],
    'mclf__estimator__min_samples_split': [4, 6, 8],
    'mclf__estimator__n_estimators': [100, 200]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)
cv.fit(X_train, y_train)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.8; total time=  39.7s
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.8; total time=  39.6s
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.8; total time=  39.6s
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.8; total time=  39.7s
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.8; total time=  39.5s
[CV] END mclf__estimator__max_depth=5, mclf__estimator__min_samples_split=4, mclf__estimator__n_estimators=100, vect__max_df=0.9; total time=  39.0s
[CV] END mclf__estimator__max_depth=5, mclf_

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x14e4a28b0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('mclf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                                                              n_jobs=-1))]),
             param_grid={'mclf__estimator__max_depth': [5, 50],
                         'mclf__estimator__min_samples_split': [4, 6, 8],
                         'mclf__estimator__n_estimators': [100, 200],
                         'vect__max_df': (0.8, 0.9)},
             verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
y_pred = cv.predict(X_test)

In [39]:
print_scores(y_test, y_pred)
print(f'best parameter: {cv.best_params_}')

related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2096
         1.0       0.76      1.00      0.86      6497

    accuracy                           0.76      8593
   macro avg       0.38      0.50      0.43      8593
weighted avg       0.57      0.76      0.65      8593

request
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      7117
         1.0       0.00      0.00      0.00      1476

    accuracy                           0.83      8593
   macro avg       0.41      0.50      0.45      8593
weighted avg       0.69      0.83      0.75      8593

offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      8547
         1.0       0.00      0.00      0.00        46

    accuracy                           0.99      8593
   macro avg       0.50      0.50      0.50      8593
weighted avg       0.99      0.99      0.99      859

/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8489
         1.0       0.00      0.00      0.00       104

    accuracy                           0.99      8593
   macro avg       0.49      0.50      0.50      8593
weighted avg       0.98      0.99      0.98      8593

refugees
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      8318
         1.0       0.00      0.00      0.00       275

    accuracy                           0.97      8593
   macro avg       0.48      0.50      0.49      8593
weighted avg       0.94      0.97      0.95      8593

death
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      8231
         1.0       0.00      0.00      0.00       362

    accuracy                           0.96      8593
   macro avg       0.48      0.50      0.49      8593
weighted avg       0.92      0.96      0.94      8593

othe

/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/christianschroeder/.local/share/virtualenvs/backend-lks6GYUP/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [40]:
disaster_response_model = 'DisasterResponseModel.pickle'
pickle.dump(cv, open(disaster_response_model, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.